### **Phase 2: Ball Tracking and Player Statistics with DEEPSORT** 🎥⚽


1. **Player & Ball Tracking** ⚽

In [ ]:
# test with cli command
!yolo track model="/Users/alyazouzou/Desktop/CV_Football/FootCVision/phase1/runs/detect/train/weights/best.pt" source="/Users/alyazouzou/Desktop/CV_Football/vids/far.mov" conf=0.3 iou=0.5 show

In [1]:
import cv2
from track import PlayerTracker

**Tracking on a frame**

In [26]:
import cv2
import numpy as np
import pandas as pd
from collections import defaultdict
from ultralytics import YOLO

video_path = "/Users/alyazouzou/Desktop/CV_Football/vids/small.mov"  # Replace with your actual video path
path = "/Users/alyazouzou/Desktop/CV_Football/FootCVision/phase1/runs/detect/train/weights/best.pt"

# Load the YOLO model
model = YOLO(path)
cap = cv2.VideoCapture(video_path)

# To store tracking information
track_history = defaultdict(lambda: [])
tracking_data = []  # List to store all tracking data for DataFrame

frame_number = 0  # Initialize frame number counter

# Class labels for the specific task
class_labels = {0: "ball", 1: "goalkeeper", 2: "player", 3: "referee"}

while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Get the dimensions of the frame (image width and height)
        image_height, image_width, _ = frame.shape
        
        # Increment frame number
        frame_number += 1

        # Track objects
        results = model.track(frame, persist=True, tracker="bytetrack.yaml")
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        classes = results[0].boxes.cls.int().cpu().tolist()  # Get class labels for each box

        for box, track_id, class_id in zip(boxes, track_ids, classes):
            x, y, w, h = box
            # Convert xywh to normalized bounding box coordinates (x1, y1, x2, y2)
            x1, y1, x2, y2 = x / image_width, y / image_height, (x + w) / image_width, (y + h) / image_height
            
            # Track history for each object
            track = track_history[track_id]
            track.append((float(x1), float(y1), float(x2), float(y2)))

            # Only keep the last 30 points for tracking
            if len(track) > 30:
                track.pop(0)

            # Get class label from the class_id
            class_name = class_labels.get(class_id, "unknown")

            # Store the tracking data for each object in the DataFrame
            for point in track:
                tracking_data.append({
                    "frame": frame_number,
                    "track_id": track_id,
                    "class": class_name,
                    "x1": point[0],
                    "y1": point[1],
                    "x2": point[2],
                    "y2": point[3]
                })

    else:
        break

# Release the video capture
cap.release()

# Create DataFrame from the tracking data
df = pd.DataFrame(tracking_data)



0: 384x640 18 players, 107.6ms
Speed: 3.8ms preprocess, 107.6ms inference, 8.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 57.3ms
Speed: 1.2ms preprocess, 57.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 players, 54.4ms
Speed: 0.9ms preprocess, 54.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 players, 85.4ms
Speed: 1.1ms preprocess, 85.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 players, 60.8ms
Speed: 1.2ms preprocess, 60.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 players, 58.6ms
Speed: 1.2ms preprocess, 58.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 players, 58.8ms
Speed: 0.9ms preprocess, 58.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 players, 56.7ms
Speed: 0.9ms preprocess, 56.7ms inference, 0.3ms postprocess per image 

In [28]:
print(df)
df.to_csv('tracking_data_with_classes.csv', index=True)

        frame  track_id   class        x1        y1        x2        y2
0           1         1  player  0.121323  0.443458  0.139888  0.526246
1           1         2  player  0.072467  0.341569  0.088945  0.420228
2           1         3  player  0.826134  0.473646  0.842901  0.557175
3           1         4  player  0.431041  0.492504  0.443683  0.576957
4           1         5  player  0.857273  0.440295  0.876691  0.522920
...       ...       ...     ...       ...       ...       ...       ...
260143    457       156  player  0.471433  0.363887  0.482797  0.444121
260144    457       156  player  0.470715  0.365001  0.482113  0.445902
260145    457       156  player  0.470475  0.365374  0.481842  0.446520
260146    457       156  player  0.470103  0.368338  0.480801  0.442988
260147    457       156  player  0.470085  0.368651  0.480870  0.442332

[260148 rows x 7 columns]


In [5]:
# Calculate previous x and y coordinates for each track_id
df['prev_x'] = df.groupby('track_id')['x'].shift(1).reset_index(drop=True)
df['prev_y'] = df.groupby('track_id')['y'].shift(1).reset_index(drop=True)

# Calculate Euclidean distance between consecutive points
df['distance'] = np.sqrt((df['x'] - df['prev_x'])**2 + (df['y'] - df['prev_y'])**2)

# Fill NaN values in distance (first frame for each track_id)
df['distance'] = df['distance'].fillna(0)

# Calculate total distance traveled and average speed for each player
player_stats = df[df['class'] == 'player'].groupby('track_id').agg(
    total_distance=('distance', 'sum'),
    average_speed=('distance', 'mean')
).reset_index()

# Determine ball possession
ball_df = df[df['class'] == 'ball']
player_df = df[df['class'] == 'player']

# Merge ball and player data on frame
merged_df = pd.merge(ball_df, player_df, on='frame', suffixes=('_ball', '_player'))

# Calculate distance between ball and each player
merged_df['distance_to_ball'] = np.sqrt(
    (merged_df['x_ball'] - merged_df['x_player'])**2 + 
    (merged_df['y_ball'] - merged_df['y_player'])**2
)

# Find the closest player to the ball at each frame
closest_player = merged_df.loc[merged_df.groupby('frame')['distance_to_ball'].idxmin()]
closest_player = closest_player[['frame', 'track_id_player']].rename(columns={'track_id_player': 'track_id'})

# Merge closest player info back to the original DataFrame
df = pd.merge(df, closest_player, on='frame', how='left')
df['ball_possession'] = df['track_id'] == df['track_id_y']

# Clean up the DataFrame
df = df.drop(columns=['prev_x', 'prev_y', 'track_id_y'])

# Save the updated DataFrame to a CSV file
df.to_csv('tracking_data_with_stats_and_possession.csv', index=False)

# Print player stats
print(player_stats)

TypeError: incompatible index of inserted column with frame index

2. **Statistics** 📈
   - Extract metrics for each player, such as:
     - Distance covered.
     - Ball possession time.
     - Speed and acceleration.